In [1]:
# Dependencies
### Data Cleaning
import csv
import pandas as pd
import numpy as np

### Reflecting DB
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import func
Base = declarative_base()


In [2]:
# Read Tab seperated CCES Raw data (using pd.read_csv method) into a pandas Data Frame

file = "data_raw/CCES16_Common_OUTPUT_Feb2018_VV-subset.tab"
file_df = pd.read_csv(file, sep='\t')
file_df.head(2)

V101  lookupzip  inputstate  countyfips  birthyr  gender  educ  \
0  222168628       3442          33     33011.0     1969       2     2   
1  273691199      70118          22     22071.0     1994       2     2   

   CC16_330a  CC16_330b  CC16_330d  CC16_330e  CC16_332a  CC16_332b  \
0        1.0        2.0        2.0        2.0        1.0        2.0   
1        1.0        1.0        2.0        1.0        1.0        1.0   

   CC16_332c  CC16_332d  CC16_332e  CC16_335  
0        1.0        2.0        2.0       1.0  
1        1.0        1.0        1.0       1.0

In [3]:
# Read CSV file containing State strings with matching V101 numbers

StatesCSV = 'data_raw/state_names.csv'
StatesDf = pd.read_csv(StatesCSV)
StatesDf.head(2)

V101      StateName
0  222168628  New Hampshire
1  273691199      Louisiana

In [4]:
# Merge the two Pandas Data Frames on V101

FinalDF = pd.merge(file_df, StatesDf, on = 'V101')
FinalDF.head(2)

V101  lookupzip  inputstate  countyfips  birthyr  gender  educ  \
0  222168628       3442          33     33011.0     1969       2     2   
1  273691199      70118          22     22071.0     1994       2     2   

   CC16_330a  CC16_330b  CC16_330d  CC16_330e  CC16_332a  CC16_332b  \
0        1.0        2.0        2.0        2.0        1.0        2.0   
1        1.0        1.0        2.0        1.0        1.0        1.0   

   CC16_332c  CC16_332d  CC16_332e  CC16_335      StateName  
0        1.0        2.0        2.0       1.0  New Hampshire  
1        1.0        1.0        1.0       1.0      Louisiana

In [5]:
# Clean the merged Data:

# Drop all rows that have an NA in any column
file_df = FinalDF.dropna()

# Reset the index so there are not missing index values
file_df1 = file_df.reset_index()

# drop second index
file_df1 = file_df1.drop(columns=['index', 'inputstate'])

file_df1.head(2)

V101  lookupzip  countyfips  birthyr  gender  educ  CC16_330a  \
0  222168628       3442     33011.0     1969       2     2        1.0   
1  273691199      70118     22071.0     1994       2     2        1.0   

   CC16_330b  CC16_330d  CC16_330e  CC16_332a  CC16_332b  CC16_332c  \
0        2.0        2.0        2.0        1.0        2.0        1.0   
1        1.0        2.0        1.0        1.0        1.0        1.0   

   CC16_332d  CC16_332e  CC16_335      StateName  
0        2.0        2.0       1.0  New Hampshire  
1        1.0        1.0       1.0      Louisiana

In [6]:
# Rename columns with more descriptive titles

file_dfRename = file_df1.rename(columns = {
    'CC16_330a':'GunBackgroundChecks_16',
    'CC16_330b':'ProhibitPublication_16',
    'CC16_330d':'BanAssultWeapons_16',
    'CC16_330e':'MakeCCPEasier_16',
    'CC16_332a':'AlwaysAllowChoice_16',
    'CC16_332b':'RapeIncestorHealth_16',
    'CC16_332c':'ProhibitMoreThan20Weeks_16',
    'CC16_332d':'EmployersDeclineBenefits_16',
    'CC16_332e':'ProhibitFedFunds_16',
    'CC16_335': 'GayMarriage'
})

# Reorder the columns

CCES_df = file_dfRename[['V101', 'lookupzip', 'StateName', 'countyfips', 'birthyr', 'gender', 'educ',
       'GunBackgroundChecks_16', 'ProhibitPublication_16',
       'BanAssultWeapons_16', 'MakeCCPEasier_16', 'AlwaysAllowChoice_16',
       'RapeIncestorHealth_16', 'ProhibitMoreThan20Weeks_16',
       'EmployersDeclineBenefits_16', 'ProhibitFedFunds_16', 'GayMarriage']]
CCES_df.head(2)

V101  lookupzip      StateName  countyfips  birthyr  gender  educ  \
0  222168628       3442  New Hampshire     33011.0     1969       2     2   
1  273691199      70118      Louisiana     22071.0     1994       2     2   

   GunBackgroundChecks_16  ProhibitPublication_16  BanAssultWeapons_16  \
0                     1.0                     2.0                  2.0   
1                     1.0                     1.0                  2.0   

   MakeCCPEasier_16  AlwaysAllowChoice_16  RapeIncestorHealth_16  \
0               2.0                   1.0                    2.0   
1               1.0                   1.0                    1.0   

   ProhibitMoreThan20Weeks_16  EmployersDeclineBenefits_16  \
0                         1.0                          2.0   
1                         1.0                          1.0   

   ProhibitFedFunds_16  GayMarriage  
0                  2.0          1.0  
1                  1.0          1.0

In [7]:
########## Convert some of the columns from number answers to string value responses

# Convert gender column
CCES_df['gender'] = CCES_df['gender'].apply(lambda x: 'male' if x== 1 else 'female')

# map and convert edu column
CCES_df['educ'] = CCES_df['educ'].map({
                                            1: 'NoHS',
                                            2: 'HS_Grad',
                                            3: 'Some College',
                                            4: '2-year',
                                            5: '4-year',
                                            6: 'Post-Grad'
                                          })

# CC16_330a 1 support, 2 oppose, 8 skipped / Gun Background Checks_16
CCES_df['GunBackgroundChecks_16'] = CCES_df['GunBackgroundChecks_16'].map({
    1: 'Support',
    2: 'Oppose',
    8: 'Skipped'
})

# CC16_330b 1 support, 2 oppose, 8 skipped  'CC16_330b':'Prohibit Publication_16',
CCES_df['ProhibitPublication_16'] = CCES_df['ProhibitPublication_16'].map({
    1: 'Support',
    2: 'Oppose',
    8: 'Skipped'
})

# CC16_330d 1 support, 2 oppose, 8 skipped 'CC16_330d':'Ban Assult Weapons_16',
CCES_df['BanAssultWeapons_16'] = CCES_df['BanAssultWeapons_16'].map({
    1: 'Support',
    2: 'Oppose',
    8: 'Skipped'
})

# CC16_330e 1 support, 2 oppose, 8 skipped 'CC16_330e':'Make CCP Easier_16',
CCES_df['MakeCCPEasier_16'] = CCES_df['MakeCCPEasier_16'].map({
    1: 'Support',
    2: 'Oppose',
    8: 'Skipped'
})

# CC16_332a 1 support, 2 oppose, 8 skipped 'CC16_332a':'Always Allow Choice_16',
CCES_df['AlwaysAllowChoice_16'] = CCES_df['AlwaysAllowChoice_16'].map({
    1: 'Support',
    2: 'Oppose',
    8: 'Skipped'
})

# CC16_332b 1 support, 2 oppose, 8 skipped   'CC16_332b':'Rape, Incest, or Health_16',
CCES_df['RapeIncestorHealth_16'] = CCES_df['RapeIncestorHealth_16'].map({
    1: 'Support',
    2: 'Oppose',
    8: 'Skipped'
})

# CC16_332c 1 support, 2 oppose, 8 skipped 'CC16_332c':'Prohibit MoreThan20 Weeks_16',
CCES_df['ProhibitMoreThan20Weeks_16'] = CCES_df['ProhibitMoreThan20Weeks_16'].map({
    1: 'Support',
    2: 'Oppose',
    8: 'Skipped'
})

# CC16_332d 1 support, 2 oppose, 8 skipped   'CC16_332d':'Employers decline benefits_16',
CCES_df['EmployersDeclineBenefits_16'] = CCES_df['EmployersDeclineBenefits_16'].map({
    1: 'Support',
    2: 'Oppose',
    8: 'Skipped'
})

# CC16_332e 1 support, 2 oppose, 8 skipped  'CC16_332e':'Prohibit Fed Funds_16',
CCES_df['ProhibitFedFunds_16'] = CCES_df['ProhibitFedFunds_16'].map({
    1: 'Support',
    2: 'Oppose',
    8: 'Skipped'
})

# CC16_335 1 support, 2 oppose, 8 skipped  'CC16_335': 'GayMarriage'
CCES_df['GayMarriage'] = CCES_df['GayMarriage'].map({
    1: 'Support',
    2: 'Oppose',
    8: 'Skipped'
})
CCES_df.head(2)

V101  lookupzip      StateName  countyfips  birthyr  gender     educ  \
0  222168628       3442  New Hampshire     33011.0     1969  female  HS_Grad   
1  273691199      70118      Louisiana     22071.0     1994  female  HS_Grad   

  GunBackgroundChecks_16 ProhibitPublication_16 BanAssultWeapons_16  \
0                Support                 Oppose              Oppose   
1                Support                Support              Oppose   

  MakeCCPEasier_16 AlwaysAllowChoice_16 RapeIncestorHealth_16  \
0           Oppose              Support                Oppose   
1          Support              Support               Support   

  ProhibitMoreThan20Weeks_16 EmployersDeclineBenefits_16 ProhibitFedFunds_16  \
0                    Support                      Oppose              Oppose   
1                    Support                     Support             Support   

  GayMarriage  
0     Support  
1     Support

In [8]:
## Troubleshooting to add up the total responses for each column
## If the responses for each section show a difference value of 0 - that's good news!

GunBcSupport = (len(CCES_df[CCES_df['GunBackgroundChecks_16']=='Support']))
GunBCOppose = (len(CCES_df[CCES_df['GunBackgroundChecks_16']=='Oppose']))
GunBcTotal = (len(CCES_df[CCES_df['GunBackgroundChecks_16']!='none']))
print('Gun Support Total Support: ', GunBcSupport)
print('Total Oppose: ', GunBCOppose)
print('Total Rows: ', GunBcTotal)
print('Total Difference', (GunBcTotal - GunBCOppose - GunBcSupport))
print('--------------------------------------------------------------------------')

ProhibitPubSupport = (len(CCES_df[CCES_df['ProhibitPublication_16']=='Support']))
ProhibitPubOppose = (len(CCES_df[CCES_df['ProhibitPublication_16']=='Oppose']))
ProhibitPubTotal = (len(CCES_df[CCES_df['ProhibitPublication_16']!='none']))
print('Prohibit Publication_16: ', ProhibitPubSupport)
print('Total Oppose: ', ProhibitPubOppose)
print('Total Rows: ', ProhibitPubTotal)
print('Total Difference', (ProhibitPubTotal - ProhibitPubOppose - ProhibitPubSupport))
print('--------------------------------------------------------------------------')
BanAssultSupport = (len(CCES_df[CCES_df['BanAssultWeapons_16']=='Support']))
BanAssultOppose = (len(CCES_df[CCES_df['BanAssultWeapons_16']=='Oppose']))
BanAssultTotal = (len(CCES_df[CCES_df['BanAssultWeapons_16']!='none']))
print('Gun Support Total Support: ', BanAssultSupport)
print('Total Oppose: ', BanAssultOppose)
print('Total Rows: ', BanAssultTotal)
print('Total Difference', (BanAssultTotal - BanAssultOppose - BanAssultSupport))
print('--------------------------------------------------------------------------')
#########################################################################################
CCPSupport = (len(CCES_df[CCES_df['MakeCCPEasier_16']=='Support']))
CCPOppose = (len(CCES_df[CCES_df['MakeCCPEasier_16']=='Oppose']))
CCPTotal = (len(CCES_df[CCES_df['MakeCCPEasier_16']!='none']))
print('Gun Support Total Support: ', CCPSupport)
print('Total Oppose: ', CCPOppose)
print('Total Rows: ', CCPTotal)
print('Total Difference', (CCPTotal - CCPOppose - CCPSupport))
print('--------------------------------------------------------------------------')

AlwaysSupport = (len(CCES_df[CCES_df['AlwaysAllowChoice_16']=='Support']))
AlwaysOppose = (len(CCES_df[CCES_df['AlwaysAllowChoice_16']=='Oppose']))
AlwaysTotal = (len(CCES_df[CCES_df['AlwaysAllowChoice_16']!='none']))
print('Prohibit Publication_16: ', AlwaysSupport)
print('Total Oppose: ', AlwaysOppose)
print('Total Rows: ', AlwaysTotal)
print('Total Difference', (AlwaysTotal - AlwaysOppose - AlwaysSupport))
print('--------------------------------------------------------------------------')
HealthSupport = (len(CCES_df[CCES_df['RapeIncestorHealth_16']=='Support']))
HealthOppose = (len(CCES_df[CCES_df['RapeIncestorHealth_16']=='Oppose']))
HealthTotal = (len(CCES_df[CCES_df['RapeIncestorHealth_16']!='none']))
print('Gun Support Total Support: ', HealthSupport)
print('Total Oppose: ', HealthOppose)
print('Total Rows: ', HealthTotal)
print('Total Difference', (HealthTotal - HealthOppose - HealthSupport))
print('--------------------------------------------------------------------------')

#########################################################################################
MoreThan20Support = (len(CCES_df[CCES_df['ProhibitMoreThan20Weeks_16']=='Support']))
MoreThan20Oppose = (len(CCES_df[CCES_df['ProhibitMoreThan20Weeks_16']=='Oppose']))
MoreThan20Total = (len(CCES_df[CCES_df['ProhibitMoreThan20Weeks_16']!='none']))
print('Gun Support Total Support: ', MoreThan20Support)
print('Total Oppose: ', MoreThan20Oppose)
print('Total Rows: ', MoreThan20Total)
print('Total Difference', (MoreThan20Total - MoreThan20Oppose - MoreThan20Support))
print('--------------------------------------------------------------------------')

DeclineBenSupport = (len(CCES_df[CCES_df['EmployersDeclineBenefits_16']=='Support']))
DeclineBenOppose = (len(CCES_df[CCES_df['EmployersDeclineBenefits_16']=='Oppose']))
DeclineBenTotal = (len(CCES_df[CCES_df['EmployersDeclineBenefits_16']!='none']))
print('Prohibit Publication_16: ', DeclineBenSupport)
print('Total Oppose: ', DeclineBenOppose)
print('Total Rows: ', DeclineBenTotal)
print('Total Difference', (DeclineBenTotal - DeclineBenOppose - DeclineBenSupport))
print('--------------------------------------------------------------------------')
NoFedFundsSupport = (len(CCES_df[CCES_df['ProhibitFedFunds_16']=='Support']))
NoFedFundsOppose = (len(CCES_df[CCES_df['ProhibitFedFunds_16']=='Oppose']))
NoFedFundsTotal = (len(CCES_df[CCES_df['ProhibitFedFunds_16']!='none']))
print('Gun Support Total Support: ', NoFedFundsSupport)
print('Total Oppose: ', NoFedFundsOppose)
print('Total Rows: ', NoFedFundsTotal)
print('Total Difference', (NoFedFundsTotal - NoFedFundsOppose - NoFedFundsSupport))
print('--------------------------------------------------------------------------')

#########################################################################################
GaySupport = (len(CCES_df[CCES_df['GayMarriage']=='Support']))
GayOppose = (len(CCES_df[CCES_df['GayMarriage']=='Oppose']))
GayTotal = (len(CCES_df[CCES_df['GayMarriage']!='none']))
print('GayMarriage Total Support: ', GaySupport)
print('Total Oppose: ', GayOppose)
print('Total Rows: ', GayTotal)
print('Total Difference', (GayTotal - GayOppose - GaySupport))
print('--------------------------------------------------------------------------')

GenderMale = (len(CCES_df[CCES_df['gender']=='male']))
GenderFemale = (len(CCES_df[CCES_df['gender']=='female']))
GenderTotal = (len(CCES_df[CCES_df['gender']!='none']))
print('gender: ', GenderMale)
print('Total Male: ', GenderMale)
print('Total Female: ', GenderFemale)
print('Total Rows: ', GenderTotal)
print('Total Difference', (GenderTotal - GenderFemale - GenderMale))
print('--------------------------------------------------------------------------')

# Education

NoHS = (len(CCES_df[CCES_df['educ']=='NoHS']))
HS_Grad = (len(CCES_df[CCES_df['educ']=='HS_Grad']))
SomeCollege = (len(CCES_df[CCES_df['educ']=='Some College']))
year2 = (len(CCES_df[CCES_df['educ']=='2-year']))
year4= (len(CCES_df[CCES_df['educ']=='4-year']))
PostGrad = (len(CCES_df[CCES_df['educ']=='Post-Grad']))
eduTotal = (len(CCES_df[CCES_df['educ']!='none']))
print('educ Total Support: ' )
print('Total NoHS: ',NoHS )
print('Total HS_Grad: ', HS_Grad)
print('Total SomeCollege: ', SomeCollege)
print('Total year2: ', year2)
print('Total year4: ', year4)
print('Total PostGrad: ', PostGrad)
print('Total Rows: ', eduTotal)
print('Total Difference', (eduTotal - NoHS - HS_Grad- SomeCollege - year2 - year4 - PostGrad ))
print('--------------------------------------------------------------------------')


Gun Support Total Support:  56810
Total Oppose:  6071
Total Rows:  62881
Total Difference 0
--------------------------------------------------------------------------
Prohibit Publication_16:  37098
Total Oppose:  25783
Total Rows:  62881
Total Difference 0
--------------------------------------------------------------------------
Gun Support Total Support:  41793
Total Oppose:  21088
Total Rows:  62881
Total Difference 0
--------------------------------------------------------------------------
Gun Support Total Support:  23663
Total Oppose:  39218
Total Rows:  62881
Total Difference 0
--------------------------------------------------------------------------
Prohibit Publication_16:  38869
Total Oppose:  24012
Total Rows:  62881
Total Difference 0
--------------------------------------------------------------------------
Gun Support Total Support:  28768
Total Oppose:  34113
Total Rows:  62881
Total Difference 0
------------------------------------------------------------------------

In [9]:
# Create the engine to connect to a sqlite file.
# The file will not be created until the class, base, and metadata are executed

engine = create_engine("sqlite:///db/CCES_Ver50.sqlite")

# Create a variable to manage session.queries later on...
session = Session(engine)

In [10]:
# Display Dataframe Column types to ensure the class is created correctly

CCES_df.dtypes

V101                             int64
lookupzip                        int64
StateName                       object
countyfips                     float64
birthyr                          int64
gender                          object
educ                            object
GunBackgroundChecks_16          object
ProhibitPublication_16          object
BanAssultWeapons_16             object
MakeCCPEasier_16                object
AlwaysAllowChoice_16            object
RapeIncestorHealth_16           object
ProhibitMoreThan20Weeks_16      object
EmployersDeclineBenefits_16     object
ProhibitFedFunds_16             object
GayMarriage                     object
dtype: object

In [11]:
### Define our Survey Class for the sqlite db table


class Survey(Base):
    __tablename__ = 'CCES_16'
    index = Column(Integer, primary_key=True)
    V101 = Column(Integer)
    lookupzip = Column(Integer)
    StateName = Column(String(255))
    countyfips = Column(Float)
  
    birthyr = Column(Integer)
    gender = Column(String(255))
    educ = Column(String(255))
   
    GunBackgroundChecks_16 = Column(String(255))
    ProhibitPublication_16 = Column(String(255))
    BanAssultWeapons_16 = Column(String(255))
    MakeCCPEasier_16 = Column(String(255))
    AlwaysAllowChoice_16 = Column(String(255))
    RapeIncestorHealth_16 = Column(String(255))
    ProhibitMoreThan20Weeks_16 = Column(String(255))
    Employersdeclinebenefits_16 = Column(String(255))
    ProhibitFedFunds_16 = Column(String(255))
    GayMarriage = Column(String(255))

In [12]:
# Create the CCES_16 table within the database
# This step actually creates the sqlite file

Base.metadata.create_all(engine)

In [13]:
# Loop to transfer/write the DF data into the sqlite file.
# The only way I could get this to work was in a loop where the notebook wasn't trying to write more than 50 rows at a time.
# I believe this is a hardware limitation of my laptop where it could not process more than 50 rows at a time in the local
# memory without the notebook becoming unresponsive. This method has worked accuratly and quickly for 10+ consecutive tests.

# It takes approximately 1 minute for this cell to run.

x1 = 0
x2 = 50

while x1 < 62881:
    
    tempDF = CCES_df.iloc[x1:x2]
    tempDF.to_sql('CCES_16', con=engine, if_exists='append')
    x1 += 50
    x2 += 50
    print('Round complete, starting', x1, 'to', x2)
    
    

Round complete, starting 50 to 100
Round complete, starting 100 to 150
Round complete, starting 150 to 200
Round complete, starting 200 to 250
Round complete, starting 250 to 300
Round complete, starting 300 to 350
Round complete, starting 350 to 400
Round complete, starting 400 to 450
Round complete, starting 450 to 500
Round complete, starting 500 to 550
Round complete, starting 550 to 600
Round complete, starting 600 to 650
Round complete, starting 650 to 700
Round complete, starting 700 to 750
Round complete, starting 750 to 800
Round complete, starting 800 to 850
Round complete, starting 850 to 900
Round complete, starting 900 to 950
Round complete, starting 950 to 1000
Round complete, starting 1000 to 1050
Round complete, starting 1050 to 1100
Round complete, starting 1100 to 1150
Round complete, starting 1150 to 1200
Round complete, starting 1200 to 1250
Round complete, starting 1250 to 1300
Round complete, starting 1300 to 1350
Round complete, starting 1350 to 1400
Round comple

Round complete, starting 11050 to 11100
Round complete, starting 11100 to 11150
Round complete, starting 11150 to 11200
Round complete, starting 11200 to 11250
Round complete, starting 11250 to 11300
Round complete, starting 11300 to 11350
Round complete, starting 11350 to 11400
Round complete, starting 11400 to 11450
Round complete, starting 11450 to 11500
Round complete, starting 11500 to 11550
Round complete, starting 11550 to 11600
Round complete, starting 11600 to 11650
Round complete, starting 11650 to 11700
Round complete, starting 11700 to 11750
Round complete, starting 11750 to 11800
Round complete, starting 11800 to 11850
Round complete, starting 11850 to 11900
Round complete, starting 11900 to 11950
Round complete, starting 11950 to 12000
Round complete, starting 12000 to 12050
Round complete, starting 12050 to 12100
Round complete, starting 12100 to 12150
Round complete, starting 12150 to 12200
Round complete, starting 12200 to 12250
Round complete, starting 12250 to 12300


Round complete, starting 21300 to 21350
Round complete, starting 21350 to 21400
Round complete, starting 21400 to 21450
Round complete, starting 21450 to 21500
Round complete, starting 21500 to 21550
Round complete, starting 21550 to 21600
Round complete, starting 21600 to 21650
Round complete, starting 21650 to 21700
Round complete, starting 21700 to 21750
Round complete, starting 21750 to 21800
Round complete, starting 21800 to 21850
Round complete, starting 21850 to 21900
Round complete, starting 21900 to 21950
Round complete, starting 21950 to 22000
Round complete, starting 22000 to 22050
Round complete, starting 22050 to 22100
Round complete, starting 22100 to 22150
Round complete, starting 22150 to 22200
Round complete, starting 22200 to 22250
Round complete, starting 22250 to 22300
Round complete, starting 22300 to 22350
Round complete, starting 22350 to 22400
Round complete, starting 22400 to 22450
Round complete, starting 22450 to 22500
Round complete, starting 22500 to 22550


Round complete, starting 31650 to 31700
Round complete, starting 31700 to 31750
Round complete, starting 31750 to 31800
Round complete, starting 31800 to 31850
Round complete, starting 31850 to 31900
Round complete, starting 31900 to 31950
Round complete, starting 31950 to 32000
Round complete, starting 32000 to 32050
Round complete, starting 32050 to 32100
Round complete, starting 32100 to 32150
Round complete, starting 32150 to 32200
Round complete, starting 32200 to 32250
Round complete, starting 32250 to 32300
Round complete, starting 32300 to 32350
Round complete, starting 32350 to 32400
Round complete, starting 32400 to 32450
Round complete, starting 32450 to 32500
Round complete, starting 32500 to 32550
Round complete, starting 32550 to 32600
Round complete, starting 32600 to 32650
Round complete, starting 32650 to 32700
Round complete, starting 32700 to 32750
Round complete, starting 32750 to 32800
Round complete, starting 32800 to 32850
Round complete, starting 32850 to 32900


Round complete, starting 42100 to 42150
Round complete, starting 42150 to 42200
Round complete, starting 42200 to 42250
Round complete, starting 42250 to 42300
Round complete, starting 42300 to 42350
Round complete, starting 42350 to 42400
Round complete, starting 42400 to 42450
Round complete, starting 42450 to 42500
Round complete, starting 42500 to 42550
Round complete, starting 42550 to 42600
Round complete, starting 42600 to 42650
Round complete, starting 42650 to 42700
Round complete, starting 42700 to 42750
Round complete, starting 42750 to 42800
Round complete, starting 42800 to 42850
Round complete, starting 42850 to 42900
Round complete, starting 42900 to 42950
Round complete, starting 42950 to 43000
Round complete, starting 43000 to 43050
Round complete, starting 43050 to 43100
Round complete, starting 43100 to 43150
Round complete, starting 43150 to 43200
Round complete, starting 43200 to 43250
Round complete, starting 43250 to 43300
Round complete, starting 43300 to 43350


Round complete, starting 52550 to 52600
Round complete, starting 52600 to 52650
Round complete, starting 52650 to 52700
Round complete, starting 52700 to 52750
Round complete, starting 52750 to 52800
Round complete, starting 52800 to 52850
Round complete, starting 52850 to 52900
Round complete, starting 52900 to 52950
Round complete, starting 52950 to 53000
Round complete, starting 53000 to 53050
Round complete, starting 53050 to 53100
Round complete, starting 53100 to 53150
Round complete, starting 53150 to 53200
Round complete, starting 53200 to 53250
Round complete, starting 53250 to 53300
Round complete, starting 53300 to 53350
Round complete, starting 53350 to 53400
Round complete, starting 53400 to 53450
Round complete, starting 53450 to 53500
Round complete, starting 53500 to 53550
Round complete, starting 53550 to 53600
Round complete, starting 53600 to 53650
Round complete, starting 53650 to 53700
Round complete, starting 53700 to 53750
Round complete, starting 53750 to 53800


Round complete, starting 62900 to 62950


In [14]:
# Declare a base using automap_base()
Base = automap_base()
Base

sqlalchemy.ext.automap.Base

In [15]:
# Use the base class to reflect the database tables
Base.prepare(engine, reflect=True)

In [16]:
# Print all of the classses mapped to the Base (this provides a list of the table names in the sqlite.db)
Base.classes.keys()

['CCES_16']

In [17]:
# Assign the CCES_16 class to a variable called 'CCES'
CCES = Base.classes.CCES_16

In [18]:
### Accuracy Test 1
# Query support for gay marriage from pandas DF and from Sqlite file

ApproveGay = session.query(CCES).filter(CCES.GayMarriage == 'Support').count()
print(f'Sqlite query of votes supporting Gay Marrige: {ApproveGay}')
print(f'Pandas DF Sort of votes supporing Gay Marriage: {GaySupport}')

Sqlite query of votes supporting Gay Marrige: 40955
Pandas DF Sort of votes supporing Gay Marriage: 40955


In [19]:
### Accuracy Test 2
# Query support for Gun Background Checks from pandas DF and Sqlite file
SupportChecks = session.query(CCES).filter(CCES.GunBackgroundChecks_16 == 'Support').count()
print(f'Sqlite support votes: {SupportChecks}')
print(f'Pandas DF support votes: {GunBcSupport}')

Sqlite support votes: 56810
Pandas DF support votes: 56810


In [20]:
### Multi Filter Test 1
# Query Males who support background checks
# Query Females who support background checks
# Total number of support background check votes in Pandas

MaleCheckSupport = session.query(CCES).filter(CCES.gender == 'male').filter(CCES.GunBackgroundChecks_16 == 'Support').count()
FemaleCheckSupport = session.query(CCES).filter(CCES.gender == 'female').filter(CCES.GunBackgroundChecks_16 == 'Support').count()

print(f'Query Male = {MaleCheckSupport}')
print(f'Query Female = {FemaleCheckSupport}')
print('Query total = ', MaleCheckSupport + FemaleCheckSupport)
print(f'Total Support = {GunBcSupport}')

Query Male = 24617
Query Female = 32193
Query total =  56810
Total Support = 56810


In [21]:
# Filter by state how many respondents were supportive of Gun Background Checks

StatesFor = session.query(CCES.StateName, func.count(CCES.GunBackgroundChecks_16)).group_by(CCES.StateName).all()
StatesFor[0:5]

[('Alabama', 764),
 ('Alaska', 109),
 ('Arizona', 1465),
 ('Arkansas', 525),
 ('California', 5860)]